# Introduction: Business Problem

In this project we will try to find an optimal location for a restaurant. Specifically, this report will be targeted to stakeholders interested in opening a Japanese restaurant in Houston, US.

Since there are lots of restaurants in Houston we will try to detect locations that are not already crowded with restaurants. We are also particularly interested in areas with no Japanese restaurants in vicinity. We would also prefer locations as close to city center as possible, assuming that first two conditions are met.

We will use our data science powers to generate a few most promissing neighborhoods based on this criteria. Advantages of each area will then be clearly expressed so that best possible final location can be chosen by stakeholders.


### Neighborhood Candidates

Let's create latitude & longitude coordinates for centroids of our candidate neighborhoods. We will create a grid of cells covering our area of interest which is aprox. 40x40 killometers centered around Houston city center.

Let's first find the latitude & longitude of Houston downtown, using Google Maps geocoding API.

In [8]:
import requests
import pandas as pd

def get_coordinates(api_key, address, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&address={}'.format(api_key, address)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        geographical_data = results[0]['geometry']['location'] # get geographical coordinates
        lat = geographical_data['lat']
        lon = geographical_data['lng']
        return [lat, lon]
    except:
        return [None, None]
    
google_api_key="AIzaSyDrRbcNepBydqYots"
address = 'Houston, TX, US'
houston_center = get_coordinates(google_api_key, address)
print('Coordinate of {}: {}'.format(address, houston_center))

Coordinate of Houston, TX, US: [29.7604267, -95.3698028]


Now let's create a grid of area candidates, equaly spaced, centered around city center and within ~20km from downtown. Our neighborhoods will be defined as circular areas with a radius of 1000 meters, so our neighborhood centers will be 2000 meters apart.

To accurately calculate distances we need to create our grid of locations in Cartesian 2D coordinate system which allows us to calculate distances in meters (not in latitude/longitude degrees). Then we'll project those coordinates back to latitude/longitude degrees to be shown on Folium map. So let's create functions to convert between WGS84 spherical coordinate system (latitude/longitude degrees) and UTM Cartesian coordinate system (X/Y coordinates in meters).


In [9]:
#!pip install shapely
import shapely.geometry

#!pip install pyproj
import pyproj

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=15, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=15, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

print('Coordinate transformation check')
print('-------------------------------')
print('Houston center longitude={}, latitude={}'.format(houston_center[1], houston_center[0]))
x, y = lonlat_to_xy(houston_center[1], houston_center[0])
print('Houston center UTM X={}, Y={}'.format(x, y))
lo, la = xy_to_lonlat(x, y)
print('Houston center longitude={}, latitude={}'.format(lo, la))

Coordinate transformation check
-------------------------------
Houston center longitude=-95.3698028, latitude=29.7604267
Houston center UTM X=270857.62618151674, Y=3294592.223890175
Houston center longitude=-95.3698028, latitude=29.760426700000004


Let's create a hexagonal grid of cells: we offset every other row, and adjust vertical row spacing so that every cell center is equally distant from all it's neighbors.

In [10]:
houston_center_x, houston_center_y = lonlat_to_xy(houston_center[1], houston_center[0]) # City center in Cartesian coordinates

k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_min = houston_center_x - 20000
x_step = 2000
y_min = houston_center_y - (int(21/k)*k*2000)/2
y_step = 2000 * k 

latitudes = []
longitudes = []
distances_from_center = []
xs = []
ys = []
for i in range(0, int(21/k)):
    y = y_min + i * y_step
    x_offset = 1000 if i%2==0 else 0
    for j in range(0, 21):
        x = x_min + j * x_step + x_offset
        distance_from_center = calc_xy_distance(houston_center_x, houston_center_y, x, y)
        if (distance_from_center <= 20001):
            lon, lat = xy_to_lonlat(x, y)
            latitudes.append(lat)
            longitudes.append(lon)
            distances_from_center.append(distance_from_center)
            xs.append(x)
            ys.append(y)

print(len(latitudes), 'candidate neighborhood centers generated.')

364 candidate neighborhood centers generated.


Let's visualize the data we have so far: city center location and candidate neighborhood centers:

In [11]:
import folium

In [12]:
map_houston = folium.Map(location=houston_center, zoom_start=11)
folium.Marker(houston_center, popup='downtowwn').add_to(map_houston)
for lat, lon in zip(latitudes, longitudes):
    #folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_houston) 
    folium.Circle([lat, lon], radius=1000, color='blue', fill=False).add_to(map_houston)
    #folium.Marker([lat, lon]).add_to(map_houston)
map_houston

OK, we now have the coordinates of centers of neighborhoods/areas to be evaluated, equally spaced (distance from every point to it's neighbors is exactly the same) and within ~6km from Downtown Houston.

Let's now use Google Maps API to get approximate addresses of those locations.


In [13]:
def get_address(api_key, latitude, longitude, verbose=False):
    try:
        url = 'https://maps.googleapis.com/maps/api/geocode/json?key={}&latlng={},{}'.format(api_key, latitude, longitude)
        response = requests.get(url).json()
        if verbose:
            print('Google Maps API JSON result =>', response)
        results = response['results']
        address = results[0]['formatted_address']
        return address
    except:
        return None

addr = get_address(google_api_key, houston_center[0], houston_center[1])
print('Reverse geocoding check')
print('-----------------------')
print('Address of [{}, {}] is: {}'.format(houston_center[0], houston_center[1], addr))

Reverse geocoding check
-----------------------
Address of [29.7604267, -95.3698028] is: 900 Bagby St, Houston, TX 77002, USA


In [14]:
print('Obtaining location addresses: ', end='')
addresses = []
for lat, lon in zip(latitudes, longitudes):
    address = get_address(google_api_key, lat, lon)
    if address is None:
        address = 'NO ADDRESS'
    address = address.replace(', USA', '') # We don't need country part of address
    addresses.append(address)
    print(' .', end='')
print(' done.')

Obtaining location addresses:  . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


Let's now place all this into a Pandas dataframe.

In [15]:
import pandas as pd

df_locations = pd.DataFrame({'Address': addresses,
                             'Latitude': latitudes,
                             'Longitude': longitudes,
                             'X': xs,
                             'Y': ys,
                             'Distance from center': distances_from_center})

df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center
0,"22 Southbelt Industrial Dr, Houston, TX 77047",29.587506,-95.427677,264857.626182,3.275540e+06,19974.984355
1,"14932 Hooper Rd, Houston, TX 77047",29.587881,-95.407043,266857.626182,3.275540e+06,19467.922334
2,"10910 S Spectrum Blvd, Houston, TX 77047",29.588254,-95.386410,268857.626182,3.275540e+06,19157.244061
3,"3902 Fellows Rd, Houston, TX 77047",29.588624,-95.365775,270857.626182,3.275540e+06,19052.558883
4,"4523 Kilnar, Houston, TX 77048",29.588990,-95.345141,272857.626182,3.275540e+06,19157.244061
5,"5741 Brookside Rd, Pearland, TX 77581",29.589353,-95.324506,274857.626182,3.275540e+06,19467.922334
6,"6522 Lesiker Rd, Pearland, TX 77581",29.589713,-95.303871,276857.626182,3.275540e+06,19974.984355
7,"5162 Glenridge Ln, Houston, TX 77053",29.602553,-95.459005,261857.626182,3.277272e+06,19519.221296
8,"4122 Knollwick Ct, Houston, TX 77053",29.602934,-95.438369,263857.626182,3.277272e+06,18681.541692
9,"202 Foxshire Ln, Houston, TX 77047",29.603312,-95.417733,265857.626182,3.277272e+06,18027.756377


...and let's now save/persist this data into local file.

In [16]:
df_locations.to_pickle('./locations.pkl') 

read location data from local file

In [17]:
#df_locations = pd.read_pickle('./locations.pkl')

Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category, but only those that are proper restaurants - coffe shops, pizza places, bakeries etc. are not direct competitors so we don't care about those. So we will include in out list only venues that have 'restaurant' in category name, and we'll make sure to detect and include all the subcategories of specific 'Japanese restaurant' category, as we need info on Italian restaurants in the neighborhood.

Foursquare credentials are defined in hidden cell bellow.


In [18]:
# Category IDs corresponding to Italian restaurants were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

japanese_restaurant_categories = [
"4bf58dd8d48988d111941735",
"55a59bace4b013909087cb0c",
"55a59bace4b013909087cb30",
"55a59bace4b013909087cb21",
"55a59bace4b013909087cb06",
"55a59bace4b013909087cb1b",
"55a59bace4b013909087cb1e",
"55a59bace4b013909087cb18",
"55a59bace4b013909087cb24",
"55a59bace4b013909087cb15",
"55a59bace4b013909087cb27",
"55a59bace4b013909087cb12",
"4bf58dd8d48988d1d2941735",
"55a59bace4b013909087cb2d",
"55a59a31e4b013909087cb00",
"55a59af1e4b013909087cb03",
"55a59bace4b013909087cb2a",
"55a59bace4b013909087cb0f",
"55a59bace4b013909087cb33",
"55a59bace4b013909087cb09",
"55a59bace4b013909087cb36"]

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'sushi', 'toyko']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', USA', '')
    return address

def get_venues_near_location(lat, lon, category, client_id, client_secret, radius=1050, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        client_id, client_secret, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [19]:
foursquare_client_id = 'V4NF3G4CJ4IRNEXP0HZCB4CK' # your Foursquare ID
foursquare_client_secret = 'BRORKDYIL4SYLMQQGXIO' # your Foursquare Secret

In [21]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found italian restaurants

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    japanese_restaurants = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=1050 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, foursquare_client_id, foursquare_client_secret, radius=1050, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_japanese = is_restaurant(venue_categories, specific_filter=japanese_restaurant_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_japanese, x, y)
                if venue_distance<=1050:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_japanese:
                    japanese_restaurants[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, japanese_restaurants, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
japanese_restaurants = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_1050.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('japanese_restaurants_1050.pkl', 'rb') as f:
        italian_restaurants = pickle.load(f)
    with open('location_restaurants_1050.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, japanese_restaurants, location_restaurants = get_restaurants(latitudes, longitudes)
    
    # Let's persists this in local file system
    with open('restaurants_1050.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('japanese_restaurants_1050.pkl', 'wb') as f:
        pickle.dump(japanese_restaurants, f)
    with open('location_restaurants_1050.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)

Obtaining venues around candidate locations: . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . . done.


In [23]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of Japanese restaurants:', len(japanese_restaurants))
print('Percentage of Italian restaurants: {:.2f}%'.format(len(japanese_restaurants) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 2117
Total number of Japanese restaurants: 95
Percentage of Italian restaurants: 4.49%
Average number of restaurants in neighborhood: 6.021978021978022


In [25]:
print('List of Japanese restaurants')
print('-----------------------')
for r in list(japanese_restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(japanese_restaurants))

List of Japanese restaurants
-----------------------
('4d1a97172a52b60c79e0f0ff', 'Ichiban Grill Super Buffet', 29.665309067941934, -95.19511522038898, '3201 Spencer Hwy, Pasadena, TX 77504, United States', 574, True, 287552.7001036691, 3283714.649793841)
('528a90db11d29661408ad21d', 'Asahi Sushi', 29.688582894750116, -95.41306505702708, '8236 Kirby Dr Ste 200 (McNee Rd.), Houston, TX 77054, United States', 860, True, 266506.82432500814, 3286714.8081556717)
('4dd05ea9fa7664b7c6d7d7ba', 'Samurai', 29.7046608, -95.4781821, 'Houston, TX 77081, United States', 984, True, 260242.19010479722, 3288630.472698142)
('4b088d7bf964a5203a0e23e3', 'Sushi Miyuki', 29.6881473268162, -95.46291701954172, '4700 Beechnut St, Houston, TX 77096, United States', 916, True, 261680.50752851754, 3286768.277613852)
('5707e70f498ec70873d1c87d', 'Express Rolls', 29.689615999999997, -95.461904, '4848 Beechnut St, Houston, TX 77096, United States', 741, True, 261782.0284535053, 3286928.999470532)
('4b1077bef964a5202

Let's now see all the collected restaurants in our area of interest on map, and let's also show Italian restaurants in different color.

In [28]:
map_houston = folium.Map(location=houston_center, zoom_start=11)
folium.Marker(houston_center, popup='Downtown').add_to(map_houston)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_japanese = res[6]
    color = 'red' if is_japanese else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_houston)
map_houston

Looking good. So now we have all the restaurants in area within few kilometers from Downtown Houston, and we know which ones are Japanese restaurants! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new Italian restaurant!

## Methodology

In this project we will direct our efforts on detecting areas of Houston that have low restaurant density, particularly those with low number of Japanese restaurants. We will limit our analysis to area ~20km around city center.

In first step we have collected the required data: location and type (category) of every restaurant within 20km from Houston downtown. We have also identified Japanese restaurants (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of 'restaurant density' across different areas of Houston - we will use heatmaps to identify a few promising areas close to center with low number of restaurants in general (and no Japanese restaurants in vicinity) and focus our attention on those areas.

In third and final step we will focus on most promising areas and within those create clusters of locations that meet some basic requirements established in discussion with stakeholders: we will take into consideration locations with no more than two restaurants in radius of 250 meters, and we want locations without Japanese restaurants in radius of 400 meters. We will present map of all such locations but also create clusters (using k-means clustering) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.


Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the number of restaurants in every area candidate:

In [29]:
location_restaurants_count = [len(res) for res in location_restaurants]

df_locations['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

df_locations.head(10)

Average number of restaurants in every area with radius=300m: 6.021978021978022


,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area
0,"22 Southbelt Industrial Dr, Houston, TX 77047",29.587506,-95.427677,264857.626182,3.275540e+06,19974.984355,0
1,"14932 Hooper Rd, Houston, TX 77047",29.587881,-95.407043,266857.626182,3.275540e+06,19467.922334,0
2,"10910 S Spectrum Blvd, Houston, TX 77047",29.588254,-95.386410,268857.626182,3.275540e+06,19157.244061,5
3,"3902 Fellows Rd, Houston, TX 77047",29.588624,-95.365775,270857.626182,3.275540e+06,19052.558883,0
4,"4523 Kilnar, Houston, TX 77048",29.588990,-95.345141,272857.626182,3.275540e+06,19157.244061,0
5,"5741 Brookside Rd, Pearland, TX 77581",29.589353,-95.324506,274857.626182,3.275540e+06,19467.922334,0
6,"6522 Lesiker Rd, Pearland, TX 77581",29.589713,-95.303871,276857.626182,3.275540e+06,19974.984355,1
7,"5162 Glenridge Ln, Houston, TX 77053",29.602553,-95.459005,261857.626182,3.277272e+06,19519.221296,3
8,"4122 Knollwick Ct, Houston, TX 77053",29.602934,-95.438369,263857.626182,3.277272e+06,18681.541692,2
9,"202 Foxshire Ln, Houston, TX 77047",29.603312,-95.417733,265857.626182,3.277272e+06,18027.756377,1


OK, now let's calculate the distance to nearest Japanese restaurant from every area candidate center (not only those within 1000m - we want distance to closest one, regardless of how distant it is).

In [31]:
distances_to_japanese_restaurant = []

for area_x, area_y in zip(xs, ys):
    min_distance = 20000
    for res in japanese_restaurants.values():
        res_x = res[7]
        res_y = res[8]
        d = calc_xy_distance(area_x, area_y, res_x, res_y)
        if d<min_distance:
            min_distance = d
    distances_to_japanese_restaurant.append(min_distance)

df_locations['Distance to Japanese restaurant'] = distances_to_japanese_restaurant

In [32]:
df_locations.head(10)

,Address,Latitude,Longitude,X,Y,Distance from center,Restaurants in area,Distance to Japanese restaurant
0,"22 Southbelt Industrial Dr, Houston, TX 77047",29.587506,-95.427677,264857.626182,3.275540e+06,19974.984355,0,11296.179837
1,"14932 Hooper Rd, Houston, TX 77047",29.587881,-95.407043,266857.626182,3.275540e+06,19467.922334,0,11180.647850
2,"10910 S Spectrum Blvd, Houston, TX 77047",29.588254,-95.386410,268857.626182,3.275540e+06,19157.244061,5,11419.723892
3,"3902 Fellows Rd, Houston, TX 77047",29.588624,-95.365775,270857.626182,3.275540e+06,19052.558883,0,11992.218360
4,"4523 Kilnar, Houston, TX 77048",29.588990,-95.345141,272857.626182,3.275540e+06,19157.244061,0,12853.657402
5,"5741 Brookside Rd, Pearland, TX 77581",29.589353,-95.324506,274857.626182,3.275540e+06,19467.922334,0,12608.611613
6,"6522 Lesiker Rd, Pearland, TX 77581",29.589713,-95.303871,276857.626182,3.275540e+06,19974.984355,1,12281.984280
7,"5162 Glenridge Ln, Houston, TX 77053",29.602553,-95.459005,261857.626182,3.277272e+06,19519.221296,3,9498.213360
8,"4122 Knollwick Ct, Houston, TX 77053",29.602934,-95.438369,263857.626182,3.277272e+06,18681.541692,2,9742.922130
9,"202 Foxshire Ln, Houston, TX 77047",29.603312,-95.417733,265857.626182,3.277272e+06,18027.756377,1,9465.381725


In [33]:
print('Average distance to closest Japanese restaurant from each area center:', df_locations['Distance to Japanese restaurant'].mean())

Average distance to closest Japanese restaurant from each area center: 6185.102184777574


Let's crete a map showing heatmap / density of restaurants and try to extract some meaningfull info from that. Also, let's show a few circles indicating distance of 1km, 2km and 3km from Downtown Houston.

In [34]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

japanese_latlons = [[res[2], res[3]] for res in japanese_restaurants.values()]

In [36]:
from folium import plugins
from folium.plugins import HeatMap

map_houston = folium.Map(location=houston_center, zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map_houston) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
folium.Circle(houston_center, radius=10000, fill=False, color='white').add_to(map_houston)
folium.Circle(houston_center, radius=15000, fill=False, color='white').add_to(map_houston)
folium.Circle(houston_center, radius=20000, fill=False, color='white').add_to(map_houston)
map_houston


Let's create another heatmap map showing heatmap/density of Japanese restaurants only.


In [37]:
map_houston = folium.Map(location=houston_center, zoom_start=11)
folium.TileLayer('cartodbpositron').add_to(map_houston) #cartodbpositron cartodbdark_matter
HeatMap(japanese_latlons).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
folium.Circle(houston_center, radius=10000, fill=False, color='white').add_to(map_houston)
folium.Circle(houston_center, radius=15000, fill=False, color='white').add_to(map_houston)
folium.Circle(houston_center, radius=20000, fill=False, color='white').add_to(map_houston)
map_houston

Most population of Houston are resident and work in east and  northeast of the city, we will narrow our region of interest to the east of Downtown

In [39]:
roi_x_min = houston_center_x - 12000
roi_y_max = houston_center_y + 2000
roi_width = 10000
roi_height = 10000
roi_center_x = roi_x_min + 5000
roi_center_y = roi_y_max - 5000
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]

map_houston = folium.Map(location=roi_center, zoom_start=12)
HeatMap(japanese_latlons).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
folium.Circle(roi_center, radius=5000, color='white', fill=True, fill_opacity=0.4).add_to(map_houston)
map_houston

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 200m appart).

In [40]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 200
y_step = 200 * k 
roi_y_min = roi_center_y - 5000

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 100 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 5001):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

2261 candidate neighborhood centers generated.


OK. Now let's calculate two most important things for each location candidate: number of restaurants in vicinity (we'll use radius of 500 meters) and distance to closest Japanese restaurant.

In [44]:
def count_restaurants_nearby(x, y, restaurants, radius=500):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 100000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_japanese_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, japanese_restaurants)
    roi_japanese_distances.append(distance)
print('done.')

Generating data on location candidates... done.


In [49]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'Restaurants nearby':roi_restaurant_counts,
                                 'Distance to Japanese restaurant':roi_japanese_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,Restaurants nearby,Distance to Japanese restaurant
0,29.686957,-95.441429,263757.626182,3.286592e+06,0,2004.096848
1,29.686995,-95.439364,263957.626182,3.286592e+06,0,2081.290598
2,29.688309,-95.452826,262657.626182,3.286765e+06,0,890.745019
3,29.688347,-95.450761,262857.626182,3.286765e+06,0,1087.964053
4,29.688385,-95.448695,263057.626182,3.286765e+06,0,1286.042329
5,29.688424,-95.446630,263257.626182,3.286765e+06,0,1484.635970
6,29.688462,-95.444565,263457.626182,3.286765e+06,0,1683.562608
7,29.688500,-95.442499,263657.626182,3.286765e+06,0,1882.716693
8,29.688538,-95.440434,263857.626182,3.286765e+06,0,1907.014972
9,29.688576,-95.438369,264057.626182,3.286765e+06,0,1914.615496


To avoids less population density, we select locations with more than 2 restaurants and less than 10 restaurants

In [62]:
good_res_count = np.array((df_roi_locations['Restaurants nearby']>=2) & (df_roi_locations['Restaurants nearby']<=10))
print('Locations with more than two restaurants nearby:', good_res_count.sum())

good_jap_distance = np.array(df_roi_locations['Distance to Japanese restaurant']>=400)
print('Locations with no Japanese restaurants within 400m:', good_jap_distance.sum())

good_locations = np.logical_and(good_res_count, good_jap_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]

Locations with more than two restaurants nearby: 516
Locations with no Japanese restaurants within 400m: 1888
Locations with both conditions met: 256


Let's see how this looks on a map.

In [64]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_houston = folium.Map(location=roi_center, zoom_start=12)
folium.TileLayer('cartodbpositron').add_to(map_houston)
HeatMap(japanese_latlons).add_to(map_houston)
folium.Circle(roi_center, radius=5000, color='white', fill=True, fill_opacity=0.6).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_houston) 
map_houston

Let's now show those good locations in a form of heatmap

In [68]:
map_houston = folium.Map(location=roi_center, zoom_start=13)
HeatMap(good_locations, radius=25).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_houston)
map_houston

Let us now cluster those locations to create centers of zones containing good locations. Those zones, their centers and addresses will be the final result of our analysis. 

In [71]:
from sklearn.cluster import KMeans

number_of_clusters = 15

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_houston = folium.Map(location=roi_center, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_houston)
HeatMap(japanese_latlons).add_to(map_houston)
folium.Circle(roi_center, radius=5000, color='white', fill=True, fill_opacity=0.4).add_to(map_houston)
folium.Marker(houston_center).add_to(map_houston)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=True, fill_opacity=0.25).add_to(map_houston) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_houston)
map_houston

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [73]:
map_houston = folium.Map(location=roi_center, zoom_start=13)
folium.Marker(houston_center).add_to(map_houston)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_houston)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_houston)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=500, color='green', fill=False).add_to(map_houston) 
map_houston

Finaly, let's reverse geocode those candidate area centers to get the addresses which can be presented to stakeholders.

In [75]:
candidate_area_addresses = []
print('==============================================================')
print('Addresses of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    addr = get_address(google_api_key, lat, lon).replace(', USA', '')
    candidate_area_addresses.append(addr)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, houston_center_x, houston_center_y)
    print('{}{} => {:.1f}km from Downtown'.format(addr, ' '*(50-len(addr)), d/1000))
    

Addresses of centers of areas recommended for further analysis

1832 Southgate Blvd, Houston, TX 77030             => 6.6km from Downtown
1601 Bering Dr, Houston, TX 77057                  => 10.6km from Downtown
2118 Mid Ln, Houston, TX 77027                     => 7.9km from Downtown
1915 Haddon St, Houston, TX 77019                  => 3.4km from Downtown
5406 Fairdale Ln, Houston, TX 77056                => 10.3km from Downtown
1909 Norfolk St, Houston, TX 77098                 => 4.8km from Downtown
218 Pine Hollow Ln, Houston, TX 77056              => 8.7km from Downtown
5315 Weslayan St, Houston, TX 77005                => 7.9km from Downtown
4626 Shetland Ln, Houston, TX 77027                => 8.8km from Downtown
4991 Laurel St, Bellaire, TX 77401                 => 11.3km from Downtown
7171 Buffalo Speedway, Houston, TX 77025           => 8.4km from Downtown
502 Sandman St, Houston, TX 77007                  => 4.1km from Downtown
Richmond Ave @ Eastside, Houston, TX 77098   

This concludes our analysis. We have created 15 addresses representing centers of zones containing locations with low number of restaurants and no Japanese restaurants nearby, all zones being fairly close to city center  Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations. Most of the zones are located in Kreuzberg and Friedrichshain boroughs, which we have identified as interesting due to being popular with tourists, fairly close to city center and well connected by public transport.